### Parallel Function Calling Notebook

This notebook shows parellel functino calling capability.
When you call Azure OpenAI - it can provide you with Mulitple calls that need to be made so you can run them in parallel.

The code is executed in your current python kernel, so you need to start a new thread in order to enable them to execute actually in parallel instead of sequentially.
This notebook walks you through doing that with Azure OpenAI Parrallel Function Calling

### Parallel Function Calling with Parallel Function Execution

In [1]:
import os
import json
from openai import AzureOpenAI
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
import time
from dotenv import load_dotenv  

In [2]:


# Load environment variables from the .env file  
load_dotenv()  

True

In [3]:
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
print(endpoint)
key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
print(api_version)
deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
print(deployment_name)

https://mmx-aoai.openai.azure.com/
2024-05-01-preview
gpt-4-1106


In [4]:
# Initialize the Azure OpenAI client
client = AzureOpenAI(
    azure_endpoint = endpoint, 
    api_key=key,  
    api_version=api_version
)

In [5]:
# Simplified weather data
WEATHER_DATA = {
    "tokyo": {"temperature": "10", "unit": "celsius"},
    "san francisco": {"temperature": "72", "unit": "fahrenheit"},
    "paris": {"temperature": "22", "unit": "celsius"}
}

# Simplified timezone data
TIMEZONE_DATA = {
    "tokyo": "Asia/Tokyo",
    "san francisco": "America/Los_Angeles",
    "paris": "Europe/Paris"
}

In [6]:
def get_current_weather(location, unit=None):
    """Get the current weather for a given location"""
    print(f"get_current_weather called with location: {location}, unit: {unit}")  
    print('Get Current Weather time' + str(datetime.now()))
    time.sleep(20)
    
    for key in WEATHER_DATA:
        if key in location:
            print(f"Weather data found for {key}")  
            weather = WEATHER_DATA[key]
            return json.dumps({
                "location": location,
                "temperature": weather["temperature"],
                "unit": unit if unit else weather["unit"]
            })
    
    print(f"No weather data found for {location}")  
    return json.dumps({"location": location, "temperature": "unknown"})

def get_current_time(location):
    print('Get Current Time time' + str(datetime.now()))
    time.sleep(10)
    """Get the current time for a given location"""
    print(f"get_current_time called with location: {location}")  
    location_lower = location.lower()
    
    for key, timezone in TIMEZONE_DATA.items():
        if key in location_lower:
            print(f"Timezone found for {key}")  
            current_time = datetime.now(ZoneInfo(timezone)).strftime("%I:%M %p")
            return json.dumps({
                "location": location,
                "current_time": current_time
            })
    
    print(f"No timezone data found for {location_lower}")  
    return json.dumps({"location": location, "current_time": "unknown"})

## Parallel Function Calling, but running Sequentially

In [7]:

def run_conversation():
    # Initial user message
    messages = [{"role": "user", "content": "What's the weather and current time in San Francisco"}]

    # Define the functions for the model
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city name, e.g. San Francisco",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_current_time",
                "description": "Get the current time in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city name, e.g. San Francisco",
                        },
                    },
                    "required": ["location"],
                },
            }
        }
    ]

    # First API call: Ask the model to use the functions
    response = client.chat.completions.create(
        model=deployment_name,
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )

    # Process the model's response
    response_message = response.choices[0].message
    messages.append(response_message)

    print("Model's response:")  
    print(response_message)  

    # Handle function calls
    toolcallCount = 0
    if response_message.tool_calls:
        print('in here')
        for tool_call in response_message.tool_calls:
            toolcallCount += 1
            print(f"Tool call {toolcallCount}")
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            print(f"Function call: {function_name}")  
            print(f"Function arguments: {function_args}")  
            
            if function_name == "get_current_weather":
                function_response = get_current_weather(
                    location=function_args.get("location"),
                    unit=function_args.get("unit")
                )
            elif function_name == "get_current_time":
                function_response = get_current_time(
                    location=function_args.get("location")
                )
            else:
                function_response = json.dumps({"error": "Unknown function"})
            
            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            })
    else:
        print("No tool calls were made by the model.")  

    print('************************************')
    print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    print(messages)
    # Second API call: Get the final response from the model
    final_response = client.chat.completions.create(
        model=deployment_name,
        messages=messages,
    )

    return final_response.choices[0].message.content, messages
    return '', messages

# Run the conversation and print the result
time1 = datetime.now()
final_message, messages = run_conversation()
time2 = datetime.now()
time_difference = time2 - time1
print('******************************************')


print(final_message)

Model's response:
ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ComOj7bLwwXhNLWwJwRqOLLG', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_NMu7tgRDkoWgLR0uhUF7oAmH', function=Function(arguments='{"location": "San Francisco"}', name='get_current_time'), type='function')])
in here
Tool call 1
Function call: get_current_weather
Function arguments: {'location': 'San Francisco', 'unit': 'celsius'}
get_current_weather called with location: San Francisco, unit: celsius
Get Current Weather time2024-09-08 15:04:30.418478
No weather data found for San Francisco
Tool call 2
Function call: get_current_time
Function arguments: {'location': 'San Francisco'}
Get Current Time time2024-09-08 15:04:50.419759
get_current_time called with location: San Francisco
Timezone found for san francisco
*****************

In [8]:
print(f"Total seconds: {time_difference.total_seconds()} seconds")

Total seconds: 34.007082 seconds


In [9]:
messages

[{'role': 'user',
  'content': "What's the weather and current time in San Francisco"},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ComOj7bLwwXhNLWwJwRqOLLG', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_NMu7tgRDkoWgLR0uhUF7oAmH', function=Function(arguments='{"location": "San Francisco"}', name='get_current_time'), type='function')]),
 {'tool_call_id': 'call_ComOj7bLwwXhNLWwJwRqOLLG',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '{"location": "San Francisco", "temperature": "unknown"}'},
 {'tool_call_id': 'call_NMu7tgRDkoWgLR0uhUF7oAmH',
  'role': 'tool',
  'name': 'get_current_time',
  'content': '{"location": "San Francisco", "current_time": "01:05 PM"}'}]

## Parallel Function Calling with Parallel Function Execution

In [10]:

from concurrent.futures import ThreadPoolExecutor, as_completed 

def handle_tool_call(tool_call):  
    function_name = tool_call.function.name  
    function_args = json.loads(tool_call.function.arguments)  
    print(f"Function call: {function_name}")  
    print(f"Function arguments: {function_args}")  
  
    if function_name == "get_current_weather":  
        return {  
            "tool_call_id": tool_call.id,  
            "role": "tool",  
            "name": function_name,  
            "content": get_current_weather(  
                location=function_args.get("location"),  
                unit=function_args.get("unit")  
            ),  
        }  
    elif function_name == "get_current_time":  
        return {  
            "tool_call_id": tool_call.id,  
            "role": "tool",  
            "name": function_name,  
            "content": get_current_time(  
                location=function_args.get("location")  
            ),  
        }  
    else:  
        return {  
            "tool_call_id": tool_call.id,  
            "role": "tool",  
            "name": function_name,  
            "content": json.dumps({"error": "Unknown function"}),  
        }  
    

def run_conversation():
    # Initial user message
    messages = [{"role": "user", "content": "What's the weather and current time in San Francisco"}]

    # Define the functions for the model
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city name, e.g. San Francisco",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_current_time",
                "description": "Get the current time in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city name, e.g. San Francisco",
                        },
                    },
                    "required": ["location"],
                },
            }
        }
    ]

    # First API call: Ask the model to use the functions
    response = client.chat.completions.create(
        model=deployment_name,
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )

    # Process the model's response
    response_message = response.choices[0].message
    messages.append(response_message)

    print("Model's response:")  
    print(response_message)  

    # Handle function calls
    toolcallCount = 0


    if response_message.tool_calls:
        with ThreadPoolExecutor() as executor:  
            future_to_tool_call = {  
                executor.submit(handle_tool_call, tool_call): tool_call  
                for tool_call in response_message.tool_calls  
            }  
    
            for future in as_completed(future_to_tool_call):  
                toolcallCount += 1  
                print(f"Tool call {toolcallCount}")  
                result = future.result()  
                messages.append(result)  
    else:  
        print("No tool calls were made by the model.") 


    print('************************************')
    
    print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    print(messages)

    # Second API call: Get the final response from the model
    final_response = client.chat.completions.create(
        model=deployment_name,
        messages=messages,
    )

    return final_response.choices[0].message.content, messages
    return '', messages

# Run the conversation and print the result
time1 = datetime.now()
final_message, messages = run_conversation()
time2 = datetime.now()
time_difference = time2 - time1
print('******************************************')


print(final_message)

Model's response:
ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ComOj7bLwwXhNLWwJwRqOLLG', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_NMu7tgRDkoWgLR0uhUF7oAmH', function=Function(arguments='{"location": "San Francisco"}', name='get_current_time'), type='function')])
Function call: get_current_weather
Function arguments: {'location': 'San Francisco', 'unit': 'celsius'}
get_current_weather called with location: San Francisco, unit: celsius
Get Current Weather time2024-09-08 15:05:04.944895
Function call: get_current_time
Function arguments: {'location': 'San Francisco'}
Get Current Time time2024-09-08 15:05:04.945892
get_current_time called with location: San Francisco
Timezone found for san francisco
Tool call 1
No weather data found for San Francisco
Tool call 2
*************************

In [11]:
print(f"Total seconds: {time_difference.total_seconds()} seconds")

Total seconds: 23.907857 seconds
